In [1]:
import numpy as np
import tensorflow as tf

import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

from utils import strip_to_melody, remove_melody


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
def inference(midi_input, melody_model, trio_model, config):
    '''
        midi_input:   the read in midi file (i.e. midi_file.read())
        melody_model: Our model that takes a melody and predicts the latent space encoding
        trio_model:   The magenta trio model that we are using to decode the latent vector
        config:       Used for data conversions. 
                      Can get by e.g. config = configs.CONFIG_MAP[model_name]
                      Might be able to just use the config in trio_model? Unsure.
    '''
    # Convert the midi to a NoteSequence
    input_note_seq = mm.midi_to_sequence_proto(midi_input)
    # mm.midi_to_sequence_proto(midi_file.read())
    
    # Take only the melody from the input, and convert it to tensors
    melody_seq    = strip_to_melody(input_note_seq)
    melody_tensor = config.data_converter.to_tensors(melody_seq)
    
    # Get the latent representation of just the melody using our trained model
    # TODO: is predict the name of the method? encode? unknown.
    latent_code = melody_model.predict(melody_tensor)
    
    # Decode the latent representation of the melody into 3 parts using Trio
    output_trio_seq = trio_model.decode(latent_code)
    
    # return output_trio_seq

    # OR we can also slice in the orignal melody
    
    # Take out only the generated accompaniment
    non_melody_seq = remove_melody(output_trio_seq)
    
    # Stitch the original melody and the new accompaniment together.
    recombined_seq = mm.concatenate_sequences(melody_seq, non_melody_seq)
    
    return recombined_seq
    

# Testing Inference

## Parameter Setup 

### Trio Model

In [3]:
model_name_melody_2bar  = 'cat-mel_2bar_big'
model_name_melody_16bar = 'hierdec-mel_16bar'
model_name_trio_16bar   = 'hierdec-trio_16bar'

model_name = model_name_trio_16bar
config = configs.CONFIG_MAP[model_name]

trio_model = TrainedModel(config,
                     batch_size=16,
                     checkpoint_dir_or_path='./models/pretrained/{}.ckpt'.format(model_name))

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, HierarchicalLstmDecoder, and hparams:
{'max_seq_len': 256, 'z_size': 512, 'free_bits': 256, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 16, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False}


I0516 14:01:39.695813 139983137478464 base_model.py:158] Building MusicVAE model with BidirectionalLstmEncoder, HierarchicalLstmDecoder, and hparams:
{'max_seq_len': 256, 'z_size': 512, 'free_bits': 256, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 16, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False}


Instructions for updating:
Colocations handled automatically by placer.


W0516 14:01:39.700670 139983137478464 deprecation.py:323] From /home/brendan/.virtualenvs/magenta/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]



...

W0516 14:01:47.469783 139983137478464 deprecation.py:323] From /home/brendan/.virtualenvs/magenta/lib/python3.6/site-packages/tensorflow/python/ops/rnn.py:626: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


W0516 14:01:48.006736 139983137478464 deprecation.py:323] From /home/brendan/.virtualenvs/magenta/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1266: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from ./models/pretrained/hierdec-trio_16bar.ckpt


I0516 14:01:48.008653 139983137478464 saver.py:1270] Restoring parameters from ./models/pretrained/hierdec-trio_16bar.ckpt


### midi

In [4]:
midi = None
with open('./data/clean_midi/The Beatles/Help!.2.mid', 'rb') as midi_file:
    midi = midi_file.read()

### Load the Melody Model

In [7]:
# TODO: load our model
melody_model = None

## Running inference

In [8]:
inference(midi, melody_model, trio_model, config)

AttributeError: 'NoneType' object has no attribute 'predict'